## 🔧 Package Setup (Run This First!)

**Important:** After updating the `fabricla_connector` package code, run the cell below to reload modules.

This ensures the notebook uses the latest version of the refactored package.

In [ ]:
# IMPORTANT: Force module reload for development
import sys
import os

# Get the workspace root (parent of notebooks folder)
workspace_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(workspace_root, 'src')

# Add to path if not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in Python path: {src_path}")

# Remove all fabricla_connector modules from cache
modules_to_remove = [key for key in sys.modules.keys() if key.startswith('fabricla_connector')]
for module in modules_to_remove:
    del sys.modules[module]
    
if modules_to_remove:
    print(f"✅ Cleared {len(modules_to_remove)} cached module(s)")
else:
    print(f"ℹ️ No cached modules to clear")

print("\n" + "="*60)
print("READY - Now run the cells below")
print("="*60)

# FabricLA Connector Framework - Quick Start Example

This notebook demonstrates how to use the consolidated FabricLA Connector framework for ingesting Fabric data into Azure Log Analytics.

## Features
- ✅ Consolidated framework with all notebook patterns
- ✅ Enhanced error handling and retry logic
- ✅ Size-aware batching for optimal performance
- ✅ Comprehensive troubleshooting and reporting
- ✅ Support for pipelines, dataflows, and capacity monitoring

In [ ]:
# === Framework Setup ===
import sys
import os

# Add framework to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import framework components - using refactored imports
from fabricla_connector.config import get_config
from fabricla_connector.api import FabricAPIClient
from fabricla_connector.collectors import PipelineDataCollector  # Updated from PipelineCollector
from fabricla_connector.collectors import DataflowCollector
from fabricla_connector.workflows import collect_and_ingest_pipeline_data_enhanced
from fabricla_connector.ingestion import AzureMonitorIngestionClient  # Updated from FabricIngestion
from fabricla_connector.utils import create_time_window, within_lookback_minutes
from dotenv import load_dotenv

print("✅ FabricLA Connector framework loaded successfully")

In [ ]:
# === Configuration ===
load_dotenv()
config = get_config()

# Basic configuration
workspace_id = config.get("FABRIC_WORKSPACE_ID")
lookback_minutes = 1440  # Last 24 hours

# Example item IDs (replace with your actual IDs)
pipeline_ids = [
    # "your-pipeline-id-here"
]

dataflow_ids = [
    # "your-dataflow-id-here" 
]

# Stream names (must match your DCR configuration)
stream_pipeline = "Custom-FabricPipelineRun_CL"
stream_activity = "Custom-FabricPipelineActivityRun_CL"
stream_dataflow = "Custom-FabricDataflowRun_CL"

print(f"📊 Configuration:")
print(f"  Workspace: {workspace_id}")
print(f"  Lookback: {lookback_minutes} minutes ({lookback_minutes/1440:.1f} days)")
print(f"  Pipeline items: {len(pipeline_ids)}")
print(f"  Dataflow items: {len(dataflow_ids)}")

In [ ]:
# === Simple Framework Usage (Recommended) ===
print("🚀 Starting data collection using framework workflow...")

try:
    # Use the high-level workflow for pipeline data
    if pipeline_ids:
        result = collect_and_ingest_pipeline_data_enhanced(
            workspace_id=workspace_id,
            pipeline_item_ids=pipeline_ids,
            lookback_minutes=lookback_minutes,
            collect_activity_runs=True,
            stream_pipeline=stream_pipeline,
            stream_activity=stream_activity,
            troubleshoot=True
        )
        
        print(f"✅ Pipeline data collected:")
        print(f"   Pipeline runs: {result.get('pipeline_runs_ingested', 0)}")
        print(f"   Activity runs: {result.get('activity_runs_ingested', 0)}")
    else:
        print("⏭️  No pipeline IDs configured")
        
    print("\n🎯 Data should appear in Log Analytics within 5-15 minutes")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# === Advanced Framework Usage (Individual Components) ===
print("🔧 Advanced usage with individual framework components...")

# Initialize components
fabric_client = FabricAPIClient()
ingestion_client = FabricIngestion()

# Manual collection with custom logic
if dataflow_ids:
    dataflow_collector = DataflowCollector(fabric_client)
    
    all_dataflow_runs = []
    for dataflow_id in dataflow_ids:
        print(f"Collecting from dataflow: {dataflow_id}")
        
        runs = dataflow_collector.collect_dataflow_runs(
            workspace_id=workspace_id,
            dataflow_id=dataflow_id,
            lookback_minutes=lookback_minutes
        )
        
        # Custom filtering (example)
        recent_runs = [
            run for run in runs
            if within_lookback_minutes(
                run.get('startTime', ''),
                run.get('endTime', ''),
                lookback_minutes
            )
        ]
        
        all_dataflow_runs.extend(recent_runs)
        print(f"  Found {len(runs)} total runs, {len(recent_runs)} recent")
    
    # Enhanced ingestion with troubleshooting
    if all_dataflow_runs:
        result = ingestion_client.ingest_enhanced(
            records=all_dataflow_runs,
            troubleshoot=True
        )
        
        print(f"\n✅ Dataflow ingestion completed:")
        print(f"   Successful: {result.get('successful_records', 0)}")
        print(f"   Failed: {result.get('failed_records', 0)}")
        print(f"   Success rate: {result.get('success_rate', 0):.1f}%")
    else:
        print("No dataflow runs to ingest")
else:
    print("No dataflow IDs configured for advanced example")

In [ ]:
# === Verification and Troubleshooting ===
print("🔍 Framework capabilities for troubleshooting:")
print("\n1. Enhanced Error Handling:")
print("   - Detailed HTTP status code handling (401, 403, 404, 429)")
print("   - Automatic retry with exponential backoff")
print("   - Comprehensive error reporting")

print("\n2. Size-Aware Batching:")
print("   - JSON size-based chunking (950KB limit)")
print("   - Automatic batch splitting for large payloads")
print("   - Performance metrics tracking")

print("\n3. Troubleshooting Reports:")
print("   - Detailed ingestion summaries")
print("   - Configuration validation")
print("   - Actionable error recommendations")

print("\n4. KQL Queries for Verification:")
print("```kql")
print("// Check table row counts")
print("FabricPipelineRun_CL | count")
print("FabricDataflowRun_CL | count")
print("")
print("// Recent data (last 24 hours)")
print("FabricPipelineRun_CL | where TimeGenerated > ago(24h) | take 10")
print("\"\"\")")

print("\n✅ Framework provides comprehensive monitoring and troubleshooting capabilities!")